Это ML-CV Телеграм бот, который реализует pipeline: получает через Телеграм бот (ROALNDbot) фото повреждения машины, отправляет его сюда в Google Colab, далее отправляет на сайт Roboflow к выбранной ML модели, получает результат анализа изображения и отправляет его обратнов Телеграм бот, ввиде ответа на картинку.
Ресурс с опенсорсными ML моделями:
- https://universe.roboflow.com/
- https://universe.roboflow.com/mario-cv/etiquetado-de-danos/images/K1oKdelXXGWIS4OlqXZF?queryText=&pageSize=50&startingIndex=0&browseQuery=true

1. УСТАНАВЛИВАЕМ ОДНУ ИЗ БИБЛИОТЕК ДЛЯ ТЕЛЕГРАМ

In [ ]:
!pip install telebot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 kB 6.0 MB/s eta 0:00:00


2. В ТЕЛЕГРАМ В БОТЕ BotFather СОЗДАЕМ СВОЕГО БОТА
- /newbot
- имя бота ROALNDbot
- получаем токен для API: ########################################

I've created my own bot in Telegram "ROALNDbot - turbinus_bot"-(created by @BotFather in Telegram):

Done! Congratulations on your new bot. You will find it at t.me/turbinus_bot. You can now add a description, about section and profile picture for your bot, see /help for a list of commands. By the way, when you've finished creating your cool bot, ping our Bot Support if you want a better username for it. Just make sure the bot is fully operational before you do this.

Use this token to access the HTTP API:
#############################################
Keep your token secure and store it safely, it can be used by anyone to control your bot.

For a description of the Bot API, see this page: https://core.telegram.org/bots/api

3. ИМПОРТИРУЕМ БИБЛИОТЕКУ И СОЗДАЕМ СУЩНОСТЬ С ТОКЕНОМ СОЗДАННОГО ТЕЛЕГРАМ БОТА "ROALNDbot"\
   ПРОВЕРЯЕМ РАБОТУ БОТА В РЕЖИМЕ "ЭХО-бот" 

!!! ДЛЯ РАБОТЫ БОТА НОДА ДОЛЖНА НЕПРЕРЫВНО ИСПОЛНЯТЬСЯ

In [ ]:
import telebot

#подключаемся к боту "ROALNDbot" используя токен, полученный в Телеграм
bot = telebot.TeleBot('##############################################')
                       
@bot.message_handler(commands=["start"]) #декоратор исполняется при получении ботом комманды "start" в Телеграм

def start(m, res=False):
    bot.send_message(m.chat.id, 'Бот запущен!')

@bot.message_handler(content_types=["text"]) #  это декортаор "ЭХО-бота" он возаращает сообщение, что ему написали, для проверки его работы

def handle_text(message):
    bot.send_message(message.chat.id, 'Вы написали: ' + message.text)

bot.polling(none_stop=True, interval=0)

bot.polling(none_stop=True, interval=0) -> запускаем процесс поллинга — постоянного опроса серверов Telegram на предмет новых сообщений и событий для бота.\
Параметр none_stop=True означает, что бот не будет останавливать поллинг при возникновении ошибок, а будет пытаться продолжать работу без перезапуска.\
Параметр interval=0 задаёт минимальную задержку между запросами к серверам Telegram, то есть бот опрашивает серверы практически без пауз.\
Таким образом, эта команда запускает бесконечный цикл получения обновлений от Telegram, позволяя боту постоянно реагировать на сообщения и команды пользователей.\
! Для более стабильной работы лучше обернуть bot.polling() в цикл с обработкой исключений, чтобы при падении соединения бот автоматически перезапускался.

4. ЗАПУСКАЕМ БОТ В РЕЖИМЕ "ЭХО-бот" + "ПОЛУЧЕНИЕ КАРТИНКИ" ИЗ ТЕЛЕГРАМ И СОХРАНЕНИЯ КАРТИНКИ В CЕССИЮ GOOGLE COLAB

In [ ]:
import telebot

# Создаем экземпляр бота
bot = telebot.TeleBot('##############################################')

# Функция, обрабатывающая команду /start
@bot.message_handler(commands=["start"])

def start(m, res=False):
    bot.send_message(m.chat.id, 'Бот запущен!')

# Получение сообщений от пользователя
@bot.message_handler(content_types=["text"])

def handle_text(message):

    bot.send_message(message.chat.id, 'Вы написали: ' + message.text)

# Функция, обрабатывающая команду получение картинки
@bot.message_handler(content_types=["photo"])
def handle_photo(message):
    bot.send_message(message.chat.id, 'Фото получено')
    photo = message.photo
    fileID = message.photo[-1].file_id
    file_info = bot.get_file(fileID)
    downloaded_file = bot.download_file(file_info.file_path)
    with open("image.jpg", 'wb') as new_file:
      new_file.write(downloaded_file)

# Запускаем поллинг бота (непрерывный опрос Телеграм)
bot.polling(none_stop=True, interval=0)

5. ЗАПУСКАЕМ БОТ В РЕЖИМЕ "ЭХО-бот" + "ПОЛУЧЕНИЕ КАРТИНКИ" + "ПЕРЕДАЧА КАРТИНКИ В ML-CV МОДЕЛЬ ROBOFLOW" + "ПОЛУЧЕНИЕ РЕЗУЛЬТАТ ОТ ROBOFLOW" + "ОТПРАВКА РЕЗУЛЬТАТА В ТЕЛЕГРАМ"

In [ ]:
!pip install cv

In [ ]:
!pip install inference_sdk #библиотека для подключения к Roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.1.0
    Uninstalling pillow-11.1.0:
      Successfully uninstalled pillow-11.1.0
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.11.0.86
    Uninstalling opencv-python-4.11.0.86:
      Successfully uninstalled opencv-python-4.11.0.86
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.13
    Uninstalling aiohttp-3.11.13:
      Successfully uninstalled aiohttp-3.11.13


In [ ]:
import telebot
from google.colab.patches import cv2_imshow
from inference_sdk import InferenceHTTPClient

# Создаем экземпляр бота
bot = telebot.TeleBot('##############################################')

# Функция, обрабатывающая команду /start
@bot.message_handler(commands=["start"])

def start(m, res=False):
    bot.send_message(m.chat.id, 'Бот запущен!')

# Получение сообщений от пользователя
@bot.message_handler(content_types=["text"])

def handle_text(message):
    bot.send_message(message.chat.id, 'Вы написали: ' + message.text)

@bot.message_handler(content_types=["photo"])
def handle_photo(message):
    bot.send_message(message.chat.id, 'Фото получено')
    photo = message.photo
    fileID = message.photo[-1].file_id
    file_info = bot.get_file(fileID)
    downloaded_file = bot.download_file(file_info.file_path)
    with open("image.jpg", 'wb') as new_file:
      new_file.write(downloaded_file)
        
      #Подключение к Roboflow через API (ключ находится на странице конкретной модели в Roboflow):
    CLIENT = InferenceHTTPClient(
          #обращаемся к сайту
          api_url="https://detect.roboflow.com",
          #обращаемся по ключу к модели
          api_key="bpBEGQKQ0Xzm4n4L3VPs"
      )
    result = CLIENT.infer("image.jpg", model_id="etiquetado-de-danos/1")#указываем выбранную модель 'etiquetado-de-danos/1'
    
    #обращаемся по ключу словаря "predictions" к результатам работы модели. И проверяем , что есть 'is' предсказания
    if result['predictions']:
        answer = "найден тип повреждения " + result['predictions'][0]['class'] + " с точностью: " + str(round((result['predictions'][0]['confidence']),2))
    else:
        answer = "повреждения не найдены"
    bot.send_message(message.chat.id, 'Результат: ' + answer)

# Запускаем поллинг бота (непрерывный опрос Телеграм)
bot.polling(none_stop=True, interval=0)

[<telebot.types.PhotoSize object at 0x7ad6fb88ea50>, <telebot.types.PhotoSize object at 0x7ad6fb772850>, <telebot.types.PhotoSize object at 0x7ad6c6ee7ad0>, <telebot.types.PhotoSize object at 0x7ad6c67d0590>, <telebot.types.PhotoSize object at 0x7ad6cbc34310>]


Что можно доделать в боте:
1. Добавить в функционал возвращение фотографии с размеченным изображением повреждения. В выходных данных модель уже дает координаты повреждения (см. документацию модели)
2. Добавить иснтукцию к боту, которая выводится при команде /help
3. Сделать словарь для перевода английских терминов Roboflow на русский язык.
